## RNN1

**Input:** Matrix of Word embeddings -> each element is an embedding for a word

**Output:** Single final output at the end of the network -> we can call it the global context vector (can test GloVe method and get it ready as another alternative function for Word2Vec), we will call it h-1



## RNN2  
#### ( input X:the sequence vector of characters X2 -> each element is a character)  
(hidden input h-1 : is the output of RNN1)
|  (output  vector of labels , Yi corresponds as a tashkela to Xi character

**Input:** Matrix of Character embeddings -> each element is an embedding for a character

**Output:** Matrix of labels -> each element is a label for a character

In [2]:
import torch
from torch import nn

In [34]:
# class RNN_Model(nn.Module):
#     def __init__(self, input_size, output_size, hidden_dim, n_layers):
#         super(RNN_Model, self).__init__()
        
#         # Defining some parameters
#         self.hidden_dim = hidden_dim
#         self.n_layers = n_layers
        
#         #Defining the layers
#         # RNN Layer
#         self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
#         # Fully connected layer
#         self.fc = nn.Linear(hidden_dim, output_size)
        
#     def forward(self, x, h_prev):
            
#             batch_size = x.size(0)
            
#             #initialize hidden state with h_prev
#             hidden = h_prev
            
#             #make sure hidden state dimensions are correct
#             hidden = hidden.view(self.n_layers, batch_size, self.hidden_dim)

#             # Passing in the input and hidden state into the model and obtaining outputs
#             out, hidden = self.rnn(x, hidden)
#             # Reshaping the outputs such that it can be fit into the fully connected layer
#             out = out.contiguous().view(-1, self.hidden_dim)
#             out = self.fc(out)
            
#             return out, hidden


class RNN_Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(RNN_Model, self).__init__()
        
        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        # Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x, h_prev):
        batch_size = x.size(0)
            
        # Initialize hidden state with h_prev
        hidden = h_prev.view(self.n_layers, batch_size, self.hidden_dim)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
            
        return out, hidden



Encoder model

In [35]:

# class Encoder(nn.Module):
#     """Encoder layer to encode a sequence to a hidden state"""

#     def __init__(self, input_size, hidden_size, num_layers, flavor):
#         """
#         Parameters
#         ----------
#         input_size: int
#         hidden_size: int
#             Number of hidden units in the RNN model
#         num_layers: int
#             Number of layers in the RNN model
#         flavor: str
#             Takes 'rnn', 'lstm', or 'gru' values.
#         """
#         super(Encoder, self).__init__()
#         self.input_size = input_size
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.flavor = flavor

#         if flavor == 'rnn':
#             self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
#         elif flavor == 'lstm':
#             self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
#         elif flavor == 'gru':
#             self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
#         else:
#             raise ValueError("Invalid flavor. Choose 'rnn', 'lstm', or 'gru'.")

#     def forward(self, x, hidden):
#         """
#         Forward pass of the encoder layer.

#         Parameters
#         ----------
#         x: torch.Tensor
#             Input sequence tensor of shape (batch_size, sequence_length, input_size)
#         hidden: torch.Tensor
#             Initial hidden state tensor of shape (num_layers, batch_size, hidden_size)

#         Returns
#         -------
#         output: torch.Tensor
#             Output tensor of shape (batch_size, sequence_length, hidden_size)
#         hidden: torch.Tensor
#             Updated hidden state tensor of shape (num_layers, batch_size, hidden_size)
#         """
#         output, hidden = self.rnn(x, hidden)
#         return output, hidden

#     def init_hidden(self, batch_size):
#         """
#         Initialize the hidden state tensor.

#         Parameters
#         ----------
#         batch_size: int
#             Size of the batch

#         Returns
#         -------
#         hidden: torch.Tensor
#             Initial hidden state tensor of shape (num_layers, batch_size, hidden_size)
#         """
#         if self.flavor == 'lstm':
#             return (torch.zeros(self.num_layers, batch_size, self.hidden_size),
#                     torch.zeros(self.num_layers, batch_size, self.hidden_size))
#         else:
#             return torch.zeros(self.num_layers, batch_size, self.hidden_size)


class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, flavor):
        super(Encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.flavor = flavor

        if flavor == 'rnn':
            self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        elif flavor == 'lstm':
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        elif flavor == 'gru':
            self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        else:
            raise ValueError("Invalid flavor. Choose 'rnn', 'lstm', or 'gru'.")

    def forward(self, x, hidden):
        output, hidden = self.rnn(x, hidden)
        return output, hidden

    def init_hidden(self, batch_size):
        if self.flavor == 'lstm':
            return (torch.zeros(self.num_layers, batch_size, self.hidden_size),
                    torch.zeros(self.num_layers, batch_size, self.hidden_size))
        else:
            return torch.zeros(self.num_layers, batch_size, self.hidden_size)


In [36]:
# class Tashkeel_Model(nn.Module):
#     """ encoder layer + RNN layer"""
#     def __init__(self, input_size, output_size, hidden_size, num_layers, flavor):
#         super(Tashkeel_Model, self).__init__()
#         self.encoder = Encoder(input_size, hidden_size, num_layers, flavor)
#         self.rnn = RNN_Model(input_size, output_size, hidden_size, num_layers)

#     def forward(self, x):
#         print("x shape is ", x.shape)
#         #forward pass of the encoder
#         h_minus_1 = self.encoder.init_hidden(x.shape[0])
#         print("h_minus_1 shape is ", h_minus_1.shape)
#         out, h_minus_1 = self.encoder(x, h_minus_1)
#         print("encoder output shape is ", h_minus_1.shape)
#         #adjust the shape of the output of the encoder
#         h_minus_1 = h_minus_1.view(h_minus_1.shape[1], h_minus_1.shape[0], h_minus_1.shape[2])
#         out, hidden = self.rnn(out, h_minus_1)
#         return out, hidden

class Tashkeel_Model(nn.Module):
    def __init__(self, word_input_size, char_input_size, output_size, hidden_size, num_layers, flavor):
        super(Tashkeel_Model, self).__init__()
        self.word_encoder = Encoder(word_input_size, hidden_size, num_layers, flavor)
        self.rnn = RNN_Model(char_input_size, output_size, hidden_size, num_layers)

    def forward(self, words, chars):
        # Forward pass of the word encoder
        h_minus_1 = self.word_encoder.init_hidden(words.shape[0])
        out, h_minus_1 = self.word_encoder(words, h_minus_1)
        # Adjust the shape of the output of the encoder
        h_minus_1 = h_minus_1.view(h_minus_1.shape[1], h_minus_1.shape[0], h_minus_1.shape[2])
        # Forward pass of the RNN
        out, hidden = self.rnn(chars, h_minus_1)
        return out, hidden


def train_tashkeel_model(model, train_loader, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):
        model.train()

        for batch_idx, (data, target) in enumerate(train_loader):
            # Move the data to the device that is used
            data = data.to(device)
            target = target.to(device)

            # Forward pass the Encoder
            output, _ = model(data)

            # Calculate the batch loss
            loss = criterion(output, target)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Print loss for progress
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))





In [ ]:
class Tashkeel_Model(nn.Module):
    """ encoder layer + RNN layer"""
    def __init__(self, encoder_in

In [37]:
# Example usage
input_size = 300  # Adjust based on the dimension of your word embeddings
hidden_size = 128
output_size = 300  # Adjust based on the number of classes for diacritics or the dimension of your embeddings
num_layers = 1
flavor = 'rnn'

tashkeel_model = Tashkeel_Model(input_size, output_size, hidden_size, num_layers, flavor)

# Define your DataLoader for training, including the input data
train_dataset = WordEmbeddingDataset("word_embed_model.bin")
#! TODO : add the target data
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

criterion = nn.MSELoss()  # Mean Squared Error loss for reconstruction
optimizer = optim.Adam(tashkeel_model.parameters(), lr=0.001)

train_tashkeel_model(tashkeel_model, train_loader, criterion, optimizer, num_epochs=10)



x shape is  torch.Size([32, 100])
h_minus_1 shape is  torch.Size([1, 32, 128])


RuntimeError: For unbatched 2-D input, hx should also be 2-D but got 3-D tensor

In [ ]:
# Example usage:
input_size = 300  # Adjust based on the dimension of your word embeddings
hidden_dim = 128
n_layers = 2  # Number of RNN layers

word_encoder = WordEncoder(input_size, hidden_dim, n_layers)

# Assuming 'input_sequence' is your input tensor
output_sequence, hidden_state = word_encoder(input_sequence)

In [6]:
import fasttext


test_input_sentence = 'السلام عليكم ورحمة الله وبركاته'

test_input_sentence_words = test_input_sentence.split(' ')
print(test_input_sentence_words)

# load the embedding model
model = fasttext.load_model('word_embedding_model.bin')

# get the embedding of the input sentence
test_input_sentence_embedding = [
    model[word] for word in test_input_sentence_words
]
print(test_input_sentence_embedding)

# convert the embedding to tensor
test_input_sentence_embedding = torch.tensor(test_input_sentence_embedding)
print(test_input_sentence_embedding)

# reshape the tensor
test_input_sentence_embedding = test_input_sentence_embedding.view(1, -1, 100)
print(test_input_sentence_embedding)

# 





100
[[ 0.042906   -0.02756045 -0.15988487 ...  0.06463267 -0.00134799
  -0.07523397]
 [-0.17331032  0.11529611 -0.34257153 ...  0.02257426  0.08717281
   0.01006125]
 [ 0.05427927 -0.08796314 -0.3034795  ...  0.06067655 -0.19971474
   0.02589632]
 ...
 [-0.01609128  0.14161943 -0.5717865  ...  0.41028932 -0.2221118
  -0.5745774 ]
 [-0.12516408  0.20515387 -0.6757796  ...  0.16081104  0.12018422
  -0.57703876]
 [-0.12468552 -0.08162142 -0.47292602 ...  0.24188764 -0.19347623
  -0.3513778 ]]


In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim

# class WordEncoder(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(WordEncoder, self).__init__()
#         self.hidden_size = hidden_size

#         # Define the Encoder layer as bidirectional LSTM
#         self.encoder = nn.LSTM(input_size, hidden_size, bidirectional=True, batch_first=True)

#         # Define the output layer for reconstruction
#         self.reconstruction_layer = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         # Forward pass through the Encoder
        
#         _, (hidden, _) = self.encoder(x)
        
#         # Concatenate the hidden states of both directions
#         hidden = torch.cat((hidden[0], hidden[1]), dim=1)

#         # Pass the concatenated vector to the reconstruction layer
#         reconstructed_x = self.reconstruction_layer(hidden)

#         return reconstructed_x

# def train_word_encoder(encoder_model, train_loader, criterion, optimizer, num_epochs=10):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     encoder_model.to(device)

#     for epoch in range(num_epochs):
#         encoder_model.train()
        
#         for batch_idx, (data, target) in enumerate(train_loader):
#             # Move the data to the device that is used
#             data = data.to(device)

#             # Forward pass the Encoder
#             output = encoder_model(data)
            
#             # Calculate the batch loss
#             loss = criterion(output, target)
            
#             # Backpropagation
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
            
#             # Print loss for progress
#             if batch_idx % 100 == 0:
#                 print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))


# # Example usage
# input_size = 300  # Adjust based on the dimension of your word embeddings
# hidden_size = 128
# output_size = 300  # Adjust based on the number of classes for diacritics or the dimension of your embeddings

# word_encoder_model = WordEncoder(input_size, hidden_size, output_size)

# # Define your DataLoader for training, including the input data

# criterion = nn.MSELoss()  # Mean Squared Error loss for reconstruction
# optimizer = optim.Adam(word_encoder_model.parameters(), lr=0.001)

# #load embeddings bin from fasttext
# import fasttext
# import numpy as np
# import torch
# from torch.utils.data import Dataset, DataLoader

# class WordEmbeddingDataset(Dataset):
#     def __init__(self, embeddings_bin_path):
#         self.embeddings = fasttext.load_model(embeddings_bin_path)

#     def __len__(self):
#         return len(self.embeddings.words)

#     def __getitem__(self, idx):
#         word = self.embeddings.words[idx]
#         embedding = torch.tensor(self.embeddings[word])

#         return embedding, word
    
# train_dataset = WordEmbeddingDataset("word_embed_model.bin")

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# train_word_encoder(word_encoder_model, train_loader, criterion, optimizer, num_epochs=10)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import fasttext
class WordEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(WordEncoder, self).__init__()
        self.hidden_size = hidden_size

        # Define the Encoder layer as bidirectional LSTM
        #TODO : FIX the bidirectional LSTM
        self.encoder = nn.LSTM(input_size, hidden_size, bidirectional=True, batch_first=True)

        # Define the output layer for reconstruction
        self.reconstruction_layer = nn.Linear(2 * hidden_size, output_size)

    def forward(self, x):
        # Forward pass through the Encoder
        _, (hidden, _) = self.encoder(x)

        # # Concatenate the hidden states of both directions
        # hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=0)  # Use dim=0 to concatenate along the batch dimension

        # Pass the concatenated vector to the reconstruction layer
        reconstructed_x = self.reconstruction_layer(hidden)

        return reconstructed_x



class WordEmbeddingDataset(Dataset):
    def __init__(self, embeddings_bin_path):
        self.embeddings = fasttext.load_model(embeddings_bin_path)

    def __len__(self):
        return len(self.embeddings.words)

    def __getitem__(self, idx):
        word = self.embeddings.words[idx]
        embedding = torch.tensor(self.embeddings[word])

        return embedding, embedding  # Using the embedding as target for reconstruction

def train_word_encoder(encoder_model, train_loader, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    encoder_model.to(device)

    for epoch in range(num_epochs):
        encoder_model.train()

        for batch_idx, (data, target) in enumerate(train_loader):
            # Move the data to the device that is used
            data, target = data.to(device), target.to(device)

            # Forward pass the Encoder
            output = encoder_model(data)

            # Calculate the batch loss
            loss = criterion(output, target)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Print loss for progress
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

# Example usage
input_size = 300  # Adjust based on the dimension of your word embeddings
hidden_size = 128
output_size = 300  # Adjust based on the number of classes for diacritics or the dimension of your embeddings

word_encoder_model = WordEncoder(input_size, hidden_size, output_size)

# Define your DataLoader for training, including the input data
train_dataset = WordEmbeddingDataset("word_embed_model.bin")
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

criterion = nn.MSELoss()  # Mean Squared Error loss for reconstruction
optimizer = optim.Adam(word_encoder_model.parameters(), lr=0.001)

train_word_encoder(word_encoder_model, train_loader, criterion, optimizer, num_epochs=10)
